# Using [Spacy](https://spacy.io/)

Import Spacy and download a Swedish model:

In [31]:
import spacy
!python -m spacy download sv_core_news_lg --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/lib/python3.11/site-packages/pgf-2.4-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/lib/python3.11/site-packages/daison-0.1-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/lib/python3.11/site-packages/gf_wordnet-0.0.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 MB 7.1 M

Load the Swedish model:

In [32]:
nlp = spacy.load("sv_core_news_lg")

Analyze some text:

In [33]:
doc = nlp("Vi har arbetat med Korp. Nu vill jag visa hur man analyserar sin egen text med Spacy.")

Divide into sentences:

In [34]:
sentences = list(doc.sentences)
len(sentences)

2

Display the corresponding dependency trees:

In [35]:
spacy.displacy.serve(sentences, style="dep", options = {"compact": True})

/home/harisont/.local/lib/python3.11/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


Iterate over the tokens that make up a sentence and print some info about it:

In [43]:
for token in sentences[1]:
    print(
        token.i,        # position in the sentence
        token.text,     # word
        token.pos_,     # POS tag
        token.head,     # dep. head (token it is referred to)
        token.head.i)   # position of the dep. head

6 Nu ADV visa 9
7 vill AUX visa 9
8 jag PRON visa 9
9 visa VERB visa 9
10 hur ADV analyserar 12
11 man PRON analyserar 12
12 analyserar VERB visa 9
13 sin PRON text 15
14 egen ADJ text 15
15 text NOUN analyserar 12
16 med ADP Spacy 17
17 Spacy PROPN analyserar 12
18 . PUNCT visa 9


Find sentences matching a "query":

In [49]:
query = lambda token: token.pos_ == "ADV"

concordances = []
for sentence in sentences:
    for (i,token) in enumerate(sentence):
        if query(token):
            concordances.append((sentence,i))

concordances

[(Nu vill jag visa hur man analyserar sin egen text med Spacy., 0),
 (Nu vill jag visa hur man analyserar sin egen text med Spacy., 4)]

A function to convert a Spacy sentence-index pair to a Python dict in the same format used by Korp for concordances:

In [58]:
def to_korp_dict(concordance, corpus="OWN"):
    sentence,i = concordance
    korp_dict = {
        "corpus": corpus, 
        "match": { # simplification: only works for 1-token queries
            "position": i,
            "start": i,
            "end": i},
        "tokens": []}
    for token in sentence:
        korp_dict["tokens"].append({
            "word": token.text,
            "pos": token.pos_,
            "ref": str(token.i),
            "dephead": str(token.head.i),
        })
    return korp_dict

to_korp_dict(concordances[0])

{'corpus': 'OWN',
 'match': {'position': 0, 'start': 0, 'end': 0},
 'tokens': [{'word': 'Nu', 'pos': 'ADV', 'ref': '6', 'dephead': '9'},
  {'word': 'vill', 'pos': 'AUX', 'ref': '7', 'dephead': '9'},
  {'word': 'jag', 'pos': 'PRON', 'ref': '8', 'dephead': '9'},
  {'word': 'visa', 'pos': 'VERB', 'ref': '9', 'dephead': '9'},
  {'word': 'hur', 'pos': 'ADV', 'ref': '10', 'dephead': '12'},
  {'word': 'man', 'pos': 'PRON', 'ref': '11', 'dephead': '12'},
  {'word': 'analyserar', 'pos': 'VERB', 'ref': '12', 'dephead': '9'},
  {'word': 'sin', 'pos': 'PRON', 'ref': '13', 'dephead': '15'},
  {'word': 'egen', 'pos': 'ADJ', 'ref': '14', 'dephead': '15'},
  {'word': 'text', 'pos': 'NOUN', 'ref': '15', 'dephead': '12'},
  {'word': 'med', 'pos': 'ADP', 'ref': '16', 'dephead': '17'},
  {'word': 'Spacy', 'pos': 'PROPN', 'ref': '17', 'dephead': '12'},
  {'word': '.', 'pos': 'PUNCT', 'ref': '18', 'dephead': '9'}]}